# OAI RAN Bug Root-Cause Analysis Workflow

This notebook implements a structured workflow for investigating bugs in the OpenAirInterface5G (OAI) RAN project. It follows the process defined in `QAprompt.json`, guiding the user from raw bug reports to root-cause analysis by cross-referencing specifications and code.

### Agent Instruction

If you are a AI coding agent, you are given access to access the entire codebase in the current folder. Also, you have access to "spec_ingestor.py" to help you make correct citation to the latest 3GPP standard. Note it is not using AI and you should consider it a grep tool with structured output. If you make any claim about 3GPP standard, you *MUST* verify it with this tool. Use it strictly follow this pattern "C:/Users/woaiw/cmd/envs/CS211/python.exe p:/openairinterface5g/spec_ingestor.py F1AP.docx 'keyword'" for F1AP and "C:/Users/woaiw/cmd/envs/CS211/python.exe p:/openairinterface5g/spec_ingestor.py NGAP.docx 'keyword'"

### Global Definitions
The workflow adheres to the following definitions and enumerations:

```json
  "$defs": {
    "RepoEnum": {
      "type": "string",
      "enum": ["https://gitlab.eurecom.fr/oai/openairinterface5g/-/tree/develop/openair2/F1AP?ref_type=heads"]
    },
    "InterfaceEnum": {
      "type": "string",
      "enum": ["F1AP","E1AP","NGAP","XnAP","RRC","PDCP","RLC","MAC","GTP-U","SCTP"]
    },
    "VerdictEnum": {
      "type": "string",
      "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"]
    },
    "ComponentEnum": {
      "type": "string",
      "enum": ["gNB-CU", "gNB-DU", "UE", "AMF", "UPF"]
    }
  },
```

The workflow consists of sequential steps, each broken down into:
1. **Requirements:** Definition of the step's goal, inputs, and expected outputs based on `QAprompt.json`.
2. **Results:** Placeholders for the execution results.
3. **Check:** Verification criteria to ensure the step was completed successfully.

## Step 1: Bug Ingest

### 1.1 Requirements

**Goal:** Normalize a freeform bug report into a structured bug card with interface/procedure guesses and key identifiers.

**Input:**
- `bug_text`: Bug: gNB-CU crashes when gNB-DU responds to UE Context Modification request with incorrect RNTI in IE.
Repro Steps:
1) The attacker intercepts a legitimate UE Context Modification request.
2) Modify the RNTI in the IE to an incorrect value and forward it to the gNB-CU.
3) For gNB-DUs, the attacker waits for UE release and immediately sends a DLRRCMessage Transfer.

**Role & Instructions:**
Act as **BugCardBuilder**. Extract and normalize from `bug_text`: likely interface(s), procedure(s), component roles, and key IDs (transaction IDs, CU/DU UE F1AP IDs, DU ID, RNTI, PCI, served-cell list). Summarize observed vs expected (if implied).

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["bug_card"],
        "properties": {
          "bug_card": {
            "type": "object",
            "required": ["observed_behavior", "interface_guess", "procedure_guess", "components_involved"],
            "properties": {
              "observed_behavior": { "type": "string" },
              "interface_guess": { "type": "array", "items": {"$ref":"#/$defs/InterfaceEnum" } },
              "procedure_guess": { "type": "array", "items": { "type": "string" } },
              "components_involved": { "type": "array", "items": {"$ref":"#/$defs/ComponentEnum" },"minItems":1 },
              "key_ids": {
                "type": "object",
                "properties": {
                  "transaction_id": { "type": "string" },
                  "cu_ue_f1ap_id": { "type": "string" },
                  "du_ue_f1ap_id": { "type": "string" },
                  "du_id": { "type": "string" },
                  "rnti": { "type": "string" },
                  "pci": { "type": "string" },
                  "served_cell_list": { "type": "array", "items": { "type": "string" } }
                }
              },
              "signals_or_timers": { "type": "array", "items": { "type": "string" } }
            }
          }
        }
      }
```

### 1.2 Results

```json
{
  "bug_card": {
    "observed_behavior": "gNB-CU crashes when gNB-DU responds to UE Context Modification request with incorrect RNTI in IE.",
    "interface_guess": [
      "F1AP"
    ],
    "procedure_guess": [
      "UE Context Modification"
    ],
    "components_involved": [
      "gNB-CU",
      "gNB-DU"
    ],
    "key_ids": {
      "rnti": "Incorrect RNTI in IE"
    },
    "signals_or_timers": [
      "UE Context Modification Response"
    ]
  }
}
```

### 1.3 Check

**Verification:** Check that the output matches the JSON schema, specifically that `interface_guess` and `procedure_guess` are not empty, and no freeform prose is included.

## Step 2: Spec Section Fetcher

### 2.1 Requirements

**Goal:** Propose spec sections (procedures, timers, message formats) that likely govern the bug.

**Input:**
- `bug_card`: Structured description from Step 1.
- `spec_toc`: Table of contents with ids/titles/anchors.

**Role & Instructions:**
Act as **SpecIndexer**. Using `bug_card` and attached spec slices, identify 3–6 most relevant spec sections. Produce keywords, titles, and summarize `expected_behaviour`.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_spec_sections","expected_behaviour"],
        "properties": {
          "candidate_spec_sections": {
            "type": "array",
            "minItems": 3,
            "maxItems": 6,
            "items": {
              "type": "object",
              "required": ["Keywords", "title"],
              "properties": { "Keywords": { "type": "array", "items": { "type": "string" }}, "title": { "type": "array", "items": { "type": "string" }} }
            }
          },
          "expected_behaviour": {"type": "string"}
        }
      }
```

### 2.2 Results

```json
{
  "candidate_spec_sections": [
    {
      "Keywords": [
        "UE CONTEXT MODIFICATION RESPONSE"
      ],
      "title": [
        "9.2.2.8 UE CONTEXT MODIFICATION RESPONSE"
      ]
    },
    {
      "Keywords": [
        "C-RNTI"
      ],
      "title": [
        "9.3.1.32 C-RNTI"
      ]
    },
    {
      "Keywords": [
        "UE Context Modification"
      ],
      "title": [
        "8.3.2 UE Context Modification"
      ]
    }
  ],
  "expected_behaviour": "The gNB-CU should handle the UE Context Modification Response. If C-RNTI is present (indicating a new C-RNTI allocated by gNB-DU), the gNB-CU should update its UE context. It should not crash regardless of the RNTI value."
}
```

### 2.3 Check

**Verification:** Check that 3-6 relevant sections are identified and `expected_behaviour` is summarized.

## Step 3: Code Fetcher (metadata-driven)

### 3.1 Requirements

**Goal:** Select likely source files/functions using metadata (paths, interface names, keywords, ASN.1 types, timers).

**Input:**
- `bug_card`: From Step 1.
- `candidate_spec_sections`: From Step 2.
- `repo_metadata`: Path, interface names, and known keywords.

**Role & Instructions:**
Act as **CodeLocator**. Build a prioritized list of files/functions to fetch using interface names, procedure/timer, and keywords. Plan recursion if initial fetch seems insufficient.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_code"],
        "properties": {
          "candidate_code": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["path", "function_name", "reason"],
              "properties": { "path": { "type": "string" }, "function_name": { "type": "string" }, "reason": { "type": "string" } }
            }
          }
        }
      }
```

### 3.2 Results

```json
{
  "candidate_code": [
    {
      "path": "openair2/F1AP/lib/f1ap_ue_context.c",
      "function_name": "decode_ue_context_mod_resp",
      "reason": "Decodes the F1AP UE Context Modification Response message."
    },
    {
      "path": "openair2/F1AP/f1ap_cu_ue_context_management.c",
      "function_name": "CU_handle_UE_CONTEXT_MODIFICATION_RESPONSE",
      "reason": "Handles the decoded UE Context Modification Response in CU."
    },
    {
      "path": "openair2/RRC/NR/rrc_gNB.c",
      "function_name": "rrc_CU_process_ue_context_modification_response",
      "reason": "Processes the RRC implications of the modification response."
    },
    {
      "path": "openair2/COMMON/f1ap_messages_types.h",
      "function_name": "f1ap_ue_context_mod_resp_t",
      "reason": "Structure definition for the decoded message."
    }
  ]
}
```

### 3.3 Check

**Verification:** Check that `candidate_code` is not empty and includes paths and function names.

## Step 4: Lightweight Retriever

### 4.1 Requirements

**Goal:** Retrieve high-signal snippets only from shortlisted specs and code.

**Input:**
- `candidate_spec_sections`: From Step 2.
- `candidate_code`: From Step 3.

**Role & Instructions:**
Act as **SnippetRetriever**. For each query, return top snippets with exact locations (spec id/anchor, file:line-range). Keep each snippet ≤ 120 words.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["snippets"],
        "properties": {
          "snippets": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["source", "location", "text", "kind"],
              "properties": {
                "kind": { "type": "string", "enum": ["spec", "code"] },
                "source": { "type": "string" },
                "location": { "type": "string" },
                "text": { "type": "string" }
              }
            }
          }
        }
      }
```

### 4.2 Results

```json
{
  "snippets": [
    {
      "kind": "spec",
      "source": "TS 38.473",
      "location": "9.2.2.8",
      "text": "UE CONTEXT MODIFICATION RESPONSE ... C-RNTI | O | 9.3.1.32 | C-RNTI allocated at the gNB-DU"
    },
    {
      "kind": "code",
      "source": "openair2/F1AP/lib/f1ap_ue_context.c",
      "location": "1991:2034",
      "text": "bool decode_ue_context_mod_resp(...) { ... case F1AP_ProtocolIE_ID_id_gNB_CU_UE_F1AP_ID: ... case F1AP_ProtocolIE_ID_id_gNB_DU_UE_F1AP_ID: ... // No C-RNTI handling }"
    },
    {
      "kind": "code",
      "source": "openair2/COMMON/f1ap_messages_types.h",
      "location": "568:579",
      "text": "typedef struct f1ap_ue_context_mod_resp { ... // No crnti field ... } f1ap_ue_context_mod_resp_t;"
    }
  ]
}
```

### 4.3 Check

**Verification:** Check that snippets are retrieved from both spec and code, with exact locations.

## Step 5: Event/State Sketcher (draft)

### 5.1 Requirements

**Goal:** Draft expected message sequence and 5–8 state sketch from snippets.

**Input:**
- `bug_card`: From Step 1.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **SketchMaker**. Build (1) an expected message sequence as ordered events and (2) a compact state machine (5–8 states, transitions with guards). Note assumptions and open questions.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["sequence_diagram", "state_machine", "assumptions", "questions", "review_request"],
        "properties": {
          "sequence_diagram": {
            "type": "array",
            "items": { "type": "object", "required": ["from", "to", "message"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "message": { "type": "string" }, "precond": { "type": "string" }, "postcond": { "type": "string" } } }
          },
          "state_machine": {
            "type": "object",
            "required": ["states", "transitions"],
            "properties": {
              "states": { "type": "array", "items": { "type": "string" }, "minItems": 5, "maxItems": 8 },
              "transitions": {
                "type": "array",
                "items": { "type": "object", "required": ["from", "to", "on"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "on": { "type": "string" }, "guard": { "type": "string" } } }
              }
            }
          },
          "assumptions": { "type": "array", "items": { "type": "string" } },
          "questions": { "type": "array", "items": { "type": "string" } },
          "review_request": { "type": "string", "enum": ["HumanInLoop_B_required"] }
        }
      }
```

### 5.2 Results

```json
{
  "sequence_diagram": [
    {
      "from": "gNB-CU",
      "to": "gNB-DU",
      "message": "UE Context Modification Request"
    },
    {
      "from": "gNB-DU",
      "to": "gNB-CU",
      "message": "UE Context Modification Response (with incorrect C-RNTI)"
    },
    {
      "from": "gNB-CU",
      "to": "gNB-CU",
      "message": "Crash (Handling Failure)"
    }
  ],
  "state_machine": {
    "states": [
      "CU_UE_CONTEXT_ACTIVE",
      "CU_WAIT_MOD_RESP",
      "CU_CRASH"
    ],
    "transitions": [
      {
        "from": "CU_UE_CONTEXT_ACTIVE",
        "to": "CU_WAIT_MOD_RESP",
        "on": "Send Mod Req"
      },
      {
        "from": "CU_WAIT_MOD_RESP",
        "to": "CU_CRASH",
        "on": "Receive Mod Resp (Incorrect RNTI)"
      }
    ]
  },
  "assumptions": [
    "The crash is caused by the presence of the C-RNTI IE or its specific value, although the current code analysis suggests it is ignored."
  ],
  "questions": [
    "Why does the CU crash if the code seems to ignore the IE?"
  ],
  "review_request": "HumanInLoop_B_required"
}
```

### 5.3 Check

**Verification:** Check for a sequence diagram, a state machine with 5-8 states, and any assumptions.

## Step 6: Conformance Checker (simple invariants)

### 6.1 Requirements

**Goal:** Check core invariants to propose preliminary verdict.

**Input:**
- `bug_card`: From Step 1.
- `sequence_diagram`: From Step 5.
- `state_machine`: From Step 5.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **ConformanceChecker**. Using snippets as authority, test invariants: directionality, req/resp pairing, ID binding, transaction scoping, timer preconditions. For each invariant: PASS/FAIL with cite. Propose preliminary verdict.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["invariant_checks", "preliminary_verdict", "rationale"],
        "properties": {
          "invariant_checks": {
            "type": "array",
            "items": { "type": "object", "required": ["name", "result", "evidence"], "properties": { "name": { "type": "string" }, "result": { "type": "string", "enum": ["PASS", "FAIL", "UNKNOWN"] }, "evidence": { "type": "array", "items": { "type": "string" } } } }
          },
          "preliminary_verdict": { "type": "string", "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"] },
          "rationale": { "type": "string" }
        }
      }
```

### 6.2 Results

```json
{
  "invariant_checks": [
    {
      "name": "Decode C-RNTI",
      "result": "FAIL",
      "evidence": [
        "openair2/F1AP/lib/f1ap_ue_context.c:1991"
      ]
    },
    {
      "name": "Store C-RNTI",
      "result": "FAIL",
      "evidence": [
        "openair2/COMMON/f1ap_messages_types.h:568"
      ]
    }
  ],
  "preliminary_verdict": "implementation-bug",
  "rationale": "The implementation fails to decode and store the C-RNTI provided in the UE Context Modification Response, which is a valid optional IE in the spec. This missing handling likely contributes to the reported crash or state inconsistency."
}
```

### 6.3 Check

**Verification:** Check that invariants are tested and a preliminary verdict is proposed.

## Step 7: Evidence Linker

### 7.1 Requirements

**Goal:** Bundle a compact evidence pack linking spec clauses and code lines with an expected-vs-observed chain.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **EvidenceBundler**. Assemble a minimal yet complete pack: spec refs (id/anchor), code refs (file:lines), and a single paragraph contrasting expected vs observed.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["evidence_pack"],
        "properties": {
          "evidence_pack": {
            "type": "object",
            "required": ["spec_citations", "code_citations", "expected_vs_observed"],
            "properties": {
              "spec_citations": { "type": "array", "items": { "type": "string" } },
              "code_citations": { "type": "array", "items": { "type": "string" } },
              "expected_vs_observed": { "type": "string" }
            }
          }
        }
      }
```

### 7.2 Results

```json
{
  "evidence_pack": {
    "spec_citations": [
      "3GPP TS 38.473 9.2.2.8"
    ],
    "code_citations": [
      "openair2/F1AP/lib/f1ap_ue_context.c",
      "openair2/COMMON/f1ap_messages_types.h"
    ],
    "expected_vs_observed": "Expected: C-RNTI IE in UE Context Modification Response should be decoded and stored. Observed: The IE is completely ignored by the decoder and the structure lacks a field for it."
  }
}
```

### 7.3 Check

**Verification:** Check that an evidence pack with spec and code citations and a comparison paragraph is created.

## Step 8: Report Generator

### 8.1 Requirements

**Goal:** Emit a one-page final report and request human sign-off on verdict.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `evidence_pack`: From Step 7.

**Role & Instructions:**
Act as **ReportWriter**. Produce the final JSON report with {verdict, rationale, citations, suggested_repro, risks, next_steps}. Be concise and actionable.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["report", "review_request"],
        "properties": {
          "report": {
            "type": "object",
            "required": ["verdict", "rationale", "citations", "suggested_repro", "next_steps"],
            "properties": {
              "verdict": { "type": "string", "$ref": "#/$defs/VerdictEnum" },
              "rationale": { "type": "string" },
              "citations": {
                "type": "object",
                "properties": {
                  "spec": { "type": "array", "items": { "type": "string" } },
                  "code": { "type": "array", "items": { "type": "string" } }
                }
              },
              "suggested_repro": { "type": "array", "items": { "type": "string" } },
              "risks": { "type": "array", "items": { "type": "string" } },
              "next_steps": { "type": "array", "items": { "type": "string" } }
            }
          },
          "review_request": { "type": "string", "enum": ["HumanInLoop_C_required"] }
        }
      }
```

### 8.2 Results

```json
{
  "report": {
    "verdict": "implementation-bug",
    "rationale": "The gNB-CU F1AP implementation fails to decode the C-RNTI IE in the UE Context Modification Response. While the spec allows this IE (Optional), the code ignores it. This prevents the CU from updating the C-RNTI when the DU reallocates it, potentially leading to the reported crash due to state mismatch or subsequent message failures.",
    "citations": {
      "spec": [
        "TS 38.473 9.2.2.8"
      ],
      "code": [
        "openair2/F1AP/lib/f1ap_ue_context.c"
      ]
    },
    "suggested_repro": [
      "Send UE Context Modification Response with a C-RNTI IE to the CU."
    ],
    "risks": [
      "Denial of Service (Crash)",
      "Loss of connectivity"
    ],
    "next_steps": [
      "Implement decoding of C-RNTI in f1ap_ue_context.c",
      "Add crnti field to f1ap_ue_context_mod_resp_t",
      "Update RRC to handle the new RNTI."
    ]
  },
  "review_request": "HumanInLoop_C_required"
}
```

### 8.3 Check

**Verification:** Check that the final report contains a verdict, rationale, citations, and next steps.